#### 📚 This is a [Geart Fork](https://github.com/geart891/RcloneLab/tree/geart891-patch-1), [Lastest version](https://colab.research.google.com/github/geart891/RLabClone/blob/master/blackpeal_template.ipynb)!


In [0]:
#@markdown ### Install and mount GDrive { vertical-output: true }
from IPython.display import clear_output
import os

if os.path.isfile("/usr/bin/mediainfo") != True:
  os.system('apt install mediainfo')
  os.system('pip install git+git://github.com/AWConant/jikanpy.git')
if os.path.isdir("/content/drive") != True:
  from google.colab import drive
  drive.mount('/content/drive')
clear_output(wait=True)
print('Setup is completed.')

Drive is mounted. Setup is completed.


# <img src='https://i.imgur.com/N4bdPff.png' height="30" alt="IMDB"/> **Generate IMDB template**

In [0]:
import urllib.request, json, re, requests, sys, os, time
from IPython.display import clear_output
from bs4 import BeautifulSoup

# Variables
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:21.0) Gecko/20100101 Firefox/21.0'}
api_key = '991109a2'
#@markdown ###IMDB Informations
IMDB_URL_OR_ID = 'tt0317705' #@param {type:"string"}
imdb_pattern = re.compile('(tt\d{7,8})')
youtube_pattern = re.compile('(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})')
imdb_id = re.findall(imdb_pattern, IMDB_URL_OR_ID)[0]

def get_imdb_template(imdb_id):
  imdb_url = f'http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}&r=json&plot=full'
  Path = "/content/drive/Shared drives/TDN Share Drive/18+/staci carr.mkv" #@param {type:"string"}
  Link = "123124123" #@param {type:"string"}
  Screenshot = "" #@param {type:"string"}
  Trailer = "" #@param {type:"string"}
  youtube_match = re.match(youtube_pattern, Trailer)
  if Trailer != "":
    if not youtube_match:
      clear_output()
      print('Your trailer link is invalid. Please only put in youtube link!')
      sys.exit()
    else:
      trailer_enabled = True
  else:
    trailer_enabled = False

  if Screenshot != "":
    screenshot_list = Screenshot.split(" ")
    screenshot_enabled = True
  else:
    screenshot_enabled = False

  if Path == "":
    mediainfo_enabled = False
  else:
    mediainfo_enabled = True
    get_ipython().system_raw(f"mediainfo --Logfile=/root/.nfo \"{Path}\"") 
    time.sleep(5) # Wait 5s for the nfo file to populate properly
    with open('/root/.nfo', 'r', encoding='utf-8') as nfo_file:
      nfo = nfo_file.readlines()
      del nfo[1]
      size = next(filter(lambda x: x[:43] == "File size                                : ", nfo), None)
      duration = next(filter(lambda x: x[:43] == "Duration                                 : ", nfo), None)
      nfo[1] = f"Title name                               : {os.path.splitext(os.path.basename(Path))[0].replace('.', ' ')} [{size[43:].strip()}] [{duration[43:].strip()}]\n"
      nfo = ''.join(nfo)

  if Link == "":
    print("You forgot putting in the link to download!")
    sys.exit()

  #@markdown ---
  #@markdown ###Forum Requirements
  Likes = 2 #@param {type: "number"}
  Posts = 2 #@param {type: "number"}
  Thanks = True #@param {type:"boolean"}

  def get_story_line(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}'
    r = requests.get(url, headers=headers)
    data = r.text
    soup = BeautifulSoup(data, 'html.parser')
    story_line = soup.find_all('div', class_='inline canwrap')[0].find('span').text.lstrip()
    return story_line

  # Fetch data
  with urllib.request.urlopen(imdb_url) as imdb_url:
    imdb_data = json.loads(imdb_url.read().decode())

  # Process data
  text_dump = f"[center][img width=\"350px\"]{imdb_data['Poster']}[/img]\n\
                  [url=https://www.imdb.com/title/{imdb_id}][font=tahoma][color=rgb(250, 197, 28)][b][size=7]{imdb_data['Title']} ({imdb_data['Year']})[/size][/b][/color][/font][/url]\n\
                  [url=https://www.imdb.com/title/{imdb_id}][img height=\"16px\"]https://i.imgur.com/rcSipDw.png[/img][/url][size=6][b] {imdb_data['imdbRating']}[/b]/10 --- [img height=\"16px\"]https://i.imgur.com/sEpKj3O.png[/img]{imdb_data['imdbVotes']}[/size]\
                [/center]\n\
                [hr][/hr]\
                  [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Plot Summary[/b][/color][/font][/size][/indent]\n \n \
                {get_story_line(imdb_id)}\n"
  if trailer_enabled:
    text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Trailer[/b][/color][/font][/size][/indent]\n\
                   [center][media=youtube]{youtube_match.group(6)}[/media][/center]\n"
  if screenshot_enabled:
    text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Screenshot[/b][/color][/font][/size][/indent]\n\
                   [spoiler='Click here to view screenshots']\n"
    for i in screenshot_list:
      text_dump += f"[img]{i}[/img]\n"
    text_dump += "[/spoiler]"
  text_dump += f"[hr][/hr]\
                   [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Movie Info[/b][/color][/font][/size][/indent]\n\
                 [LIST]\
                   [*][b][color=rgb(251, 160, 38)]Rating :[/b][/color] {imdb_data['Rated']}\n\
                   [*][b][color=rgb(251, 160, 38)]Genre :[/b][/color] {imdb_data['Genre']}\n\
                   [*][b][color=rgb(251, 160, 38)]Directed By :[/b][/color] {imdb_data['Director']}\n\
                   [*][b][color=rgb(251, 160, 38)]Written By :[/b][/color] {imdb_data['Writer']}\n\
                   [*][b][color=rgb(251, 160, 38)]Starring :[/b][/color] {imdb_data['Actors']}\n\
                   [*][b][color=rgb(251, 160, 38)]Release Date :[/b][/color] {imdb_data['Released']}\n\
                   [*][b][color=rgb(251, 160, 38)]Runtime :[/b][/color] {imdb_data['Runtime'] if 'Runtime' in imdb_data.keys() else 'N/A'}\n\
                   [*][b][color=rgb(251, 160, 38)]On Disk/Streaming :[/b][/color] {imdb_data['DVD']}\n\
                   [*][b][color=rgb(251, 160, 38)]Box Office :[/b][/color] {imdb_data['BoxOffice']}\n\
                   [*][b][color=rgb(251, 160, 38)]Awards :[/b][/color] {imdb_data['Awards']}\n\
                   [*][b][color=rgb(251, 160, 38)]Studio :[/b][/color] {imdb_data['Production'] if 'Production' in imdb_data.keys() else 'N/A'}\
                 [/LIST]\n"
  if mediainfo_enabled:
    text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Media Info[/b][/color][/font][/size][/indent]\n\
                   [spoiler='Click here to view Media Info'][code]{nfo}[/code][/spoiler]\n"
  text_dump += ""
  text_dump += f"[hr][/hr]\
                  [center][size=6][font=tahoma][color=rgb(97, 189, 109)][b][u]Download Link[/u][/b][/color][/font][/size]\n\
                    {'[thanks]' if Thanks else ''}\
                      {f'[likes={Likes}]' if Likes != 0 else ''}\
                        {f'[posts={Posts}]' if Posts != 0 else ''}\
                          {Link}\
                        {f'[/posts]' if Posts != 0 else ''}\
                      {f'[/likes]' if Likes != 0 else ''}\
                    {f'[/thanks]' if Thanks else ''}\
                  [/center]"

  return text_dump

print(get_imdb_template(imdb_id))

# <img src='https://i.imgur.com/e5Thsob.jpg?1' height="25" alt="MAL"/> **Generate MAL template**

In [0]:
import re, time, sys, os
from bs4 import BeautifulSoup
from IPython.display import clear_output
from jikanpy import Jikan

jikan = Jikan()

#@markdown ###Anime Information
Anime_URL_OR_ID = '' #@param {type: "string"}
mal_pattern = re.compile('(anime)?/(\d{1,7})/(.+)?')
if re.match(r'\d+', Anime_URL_OR_ID):
  mal_id = Anime_URL_OR_ID
else:
  mal_id = re.findall(mal_pattern, Anime_URL_OR_ID)[0][1]
youtube_pattern = re.compile('(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})')

def get_mal_template(mal_id):
  # Fetch data
  anime_data = jikan.anime(mal_id)
  Path = "" #@param {type:"string"}
  Link = "" #@param {type:"string"}
  Screenshot = "" #@param {type:"string"}
  if Screenshot != "":
    screenshot_list = Screenshot.split(" ")
    screenshot_enabled = True
  else:
    screenshot_enabled = False

  def get_info(anime_data, info):
    text = ''
    for i in range(len(anime_data[f'{info}'])):
      text += anime_data[info][i]['name'] + ', '
    return text[:-2]

  if Path == "":
    mediainfo_enabled = False
  else:
    mediainfo_enabled = True
    get_ipython().system_raw(f"mediainfo --Logfile=/root/.nfo \"{Path}\"") 
    time.sleep(5) # Wait 5s for the nfo file to populate properly
    with open('/root/.nfo', 'r', encoding='utf-8') as nfo_file:
      nfo = nfo_file.readlines()
      del nfo[1]
      size = next(filter(lambda x: x[:43] == "File size                                : ", nfo), None)
      duration = next(filter(lambda x: x[:43] == "Duration                                 : ", nfo), None)
      nfo[1] = f"Title name                               : {os.path.splitext(os.path.basename(Path))[0].replace('.', ' ')} [{size[43:].strip()}] [{duration[43:].strip()}]\n"
      nfo = ''.join(nfo)

  if Link == "":
    print("You forgot putting in the link to download!")
    sys.exit()

  #@markdown ---
  #@markdown ###Forum Requirements
  Likes = 2 #@param {type: "number"}
  Posts = 2 #@param {type: "number"}
  Thanks = True #@param {type:"boolean"}

  # Process data
  text_dump = f"[center]\
                  [img]{anime_data['image_url']}[/img]\n\
                  [url={anime_data['url']}][font=tahoma][color=rgb(250, 197, 28)][b][size=7]{anime_data['title']} ({anime_data['type']} - {anime_data['premiered']})[/size][/b][/color][/font][/url]\n\
                  [url={anime_data['url']}][img height=\"16px\"]https://i.imgur.com/YJgzM4V.png[/img][/url][size=6][b] {anime_data['score']}[/b]/10 --- [img height=\"16px\"]https://i.imgur.com/sEpKj3O.png[/img]{anime_data['scored_by']}[/size]
                [/center]\n\
                [hr][/hr]\
                  [indent][font=tahoma][size=6][color=rgb(250, 197, 28)][b]Synopsis[/b][/color][/size][/font][/indent]\n \n{anime_data['synopsis']}\n"
  try:
    text_dump += f"[hr][/hr]\
                    [indent][font=tahoma][size=6][color=rgb(250, 197, 28)][b]Trailer[/b][/color][/size][/font][/indent]\n\
                  [center][media=youtube]{re.match(youtube_pattern, str(anime_data['trailer_url'])).group(6)}[/media][/center]\n"
  except:
    pass
  if screenshot_enabled:
    text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Screenshot[/b][/color][/font][/size][/indent]\n\
                   [spoiler='Click here to view screenshots']\n"
    for i in screenshot_list:
      text_dump += f"[img]{i}[/img]\n"
    text_dump += "[/spoiler]"
  text_dump += f"[hr][/hr]\
                   [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Anime Info[/b][/color][/font][/size][/indent]\n\
                 [LIST]\
                   [*][b][color=rgb(251, 160, 38)]Rating :[/b][/color] {anime_data['rating']}\n\
                   [*][b][color=rgb(251, 160, 38)]Type :[/b][/color] {anime_data['type']}\n\
                   [*][b][color=rgb(251, 160, 38)]Episodes :[/b][/color] {anime_data['episodes']}\n\
                   [*][b][color=rgb(251, 160, 38)]Status :[/b][/color] {anime_data['status']}\n\
                   [*][b][color=rgb(251, 160, 38)]Aired :[/b][/color] {anime_data['aired']['string']}\n\
                   [*][b][color=rgb(251, 160, 38)]Premiered :[/b][/color] {anime_data['premiered']}\n\
                   [*][b][color=rgb(251, 160, 38)]Producers :[/b][/color] {get_info(anime_data, 'producers')}\n\
                   [*][b][color=rgb(251, 160, 38)]Studios :[/b][/color] {get_info(anime_data, 'studios')}\n\
                   [*][b][color=rgb(251, 160, 38)]Source :[/b][/color] {anime_data['source']}\n\
                   [*][b][color=rgb(251, 160, 38)]Genres :[/b][/color] {get_info(anime_data, 'genres')}\n\
                   [*][b][color=rgb(251, 160, 38)]Duration :[/b][/color] {anime_data['duration']}\
                 [/LIST]\n"  
  if mediainfo_enabled:
    text_dump += f"[hr][/hr]\
                     [indent][size=6][font=tahoma][color=rgb(243, 121, 52)][b]Media Info[/b][/color][/font][/size][/indent]\n\
                   [spoiler='Click here to view Media Info'][code]{nfo}[/code][/spoiler]\n"
  text_dump += f"[hr][/hr]\
                  [center][size=6][font=tahoma][color=rgb(97, 189, 109)][b][u]Download Link[/u][/b][/color][/font][/size]\n\
                    {'[thanks]' if Thanks else ''}\
                      {f'[likes={Likes}]' if Likes != 0 else ''}\
                        {f'[posts={Posts}]' if Posts != 0 else ''}\
                          {Link}\
                        {f'[/posts]' if Posts != 0 else ''}\
                      {f'[/likes]' if Likes != 0 else ''}\
                    {f'[/thanks]' if Thanks else ''}\
                  [/center]"

  return text_dump

print(get_mal_template(mal_id))
